In [1]:
# File system and display
import os
import shutil
import cv2
import matplotlib.pyplot as plt

# Data handling
import pandas as pd
import numpy as np

# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.6.0+cu124
CUDA available: True


In [2]:
!pip install ultralytics --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3

In [3]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Define dataset path
dataset_path = "/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset"

# List contents to verify structure
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    


Traffic Dataset/
  labels/
    val/
    train/
  images/
    val/
    test/
    train/


### preprocessing step


In [5]:
img_dir = "/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/train"
label_dir = "/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/labels/train"

image_files = sorted([f[:-4] for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')])
label_files = sorted([f[:-4] for f in os.listdir(label_dir) if f.endswith('.txt')])

print(f"Images: {len(image_files)}, Labels: {len(label_files)}")

missing_labels = set(image_files) - set(label_files)
print(f"Missing labels for: {missing_labels}")


Images: 738, Labels: 738
Missing labels for: set()


In [6]:
#insall yolov5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.54 MiB | 28.42 MiB/s, done.
Resolving deltas: 100% (11990/11990), done.
/kaggle/working/yolov5


In [7]:
yaml_content = """
path: /kaggle/input/traffic-vehicles-object-detection
train: /kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/train
val: /kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/val

nc: 7
names: ['Car', 'Number Plate', 'Blur Number Plate', 'Two Wheeler', 'Auto', 'Bus', 'Truck']
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

# Confirm it's written correctly
!cat data.yaml



path: /kaggle/input/traffic-vehicles-object-detection
train: /kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/train
val: /kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/val

nc: 7
names: ['Car', 'Number Plate', 'Blur Number Plate', 'Two Wheeler', 'Auto', 'Bus', 'Truck']


In [8]:
!find . -name "data.yaml"


./data.yaml


In [9]:
!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 30 \
  --data ./data.yaml \
  --weights yolov5s.pt \
  --name yolov5s-vehicle-detection


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-27 03:53:43.202977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750996423.423078      74 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750996423.482051      74 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&

In [10]:
!ls runs/train/yolov5s-vehicle-detection2/weights/


ls: cannot access 'runs/train/yolov5s-vehicle-detection2/weights/': No such file or directory


In [11]:
!python val.py --weights runs/train/yolov5s-vehicle-detection2/weights/best.pt \
               --data data.yaml --img 640 --task val


val: data=data.yaml, weights=['runs/train/yolov5s-vehicle-detection2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Traceback (most recent call last):
  File "/kaggle/working/yolov5/val.py", line 604, in <module>
    main(opt)
  File "/kaggle/working/yolov5/val.py", line 575, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/kaggle/working/yolov5/val.py", line 270, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
            ^^^^^^^

In [12]:
!python detect.py \
  --weights runs/train/yolov5s-vehicle-detection2/weights/best.pt \
  --source "/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/test" \
  --img 640 --conf 0.25 \
  --save-txt --project runs/detect --name test_detect --exist-ok


detect: weights=['runs/train/yolov5s-vehicle-detection2/weights/best.pt'], source=/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=test_detect, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Traceback (most recent call last):
  File "/kaggle/working/yolov5/detect.py", line 438, in <module>
    main(opt)
  File "/kaggle/working/yolov5/detect.py", line 433, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    

In [13]:
!mkdir -p /kaggle/working/test_label
!cp runs/detect/test_detect/labels/*.txt /kaggle/working/test_label/


cp: cannot stat 'runs/detect/test_detect/labels/*.txt': No such file or directory


In [14]:
!zip -r /kaggle/working/test_label.zip /kaggle/working/test_label


  adding: kaggle/working/test_label/ (stored 0%)
